<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Classical%20Transportation%20Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classical Transportation Problem

## Problem Description

The classic(al) transportation problem concerns minimizing the cost of transporting a product from sources/supplies to destinations/demands. The three inputs of the model are total units produced at each source, total units needed at each destination, and the cost to transport one unit from each source to each destination. And the objective is to minimize the total cost of transporting all units produced at sources to meet the demands at destinations. (Source: https://gistbok.ucgis.org/bok-topics/classic-transportation-problem)


**Example:** <br>
Container need to be transported from sources (Kiel, Magdeburg, Dortmund) to destinations (Hamburg, Berlin, Cologne, Munich). The available containers (supply) in Kiel, Magdeburg, and Dortmund are 45, 120, and 9, respectively. The demands in Hamburg, Berlin, Cologne, Munich are 80, 78, 47, and 55, respectively The transportation costs in tsd. Euro per container are shown in the following table:

<br>
$
\begin{array}{lrrrrrrr}
    \hline
    \text{} & \text{Hamburg} & \text{Berlin} & \text{Cologne} & \text{Munich} \\ \hline
     \text{Kiel} & 20 & 40 & 50 & 120 \\
     \text{Magdeburg} & 30 & 10 & 40 & 60 \\
     \text{Dortmund} & 45 & 40 & 10 & 50 \\
     \hline
\end{array}
$
<br><br>

How to minimize the total cost of transporting all units produced at sources to meet the demands at destinations?



## Mathematical Program

**Notation:**
\begin{array}{ll}
S & \text{Set of source/supply nodes}\\
D & \text{Set of destination/demand nodes}\\
s_i & \text{Supply at node } i \in S\\
d_i & \text{Demand at node } j \in D\\
c_{ij}  & \text{Transportation costs between supply node $i \in S$ and demand node $j \in D$}\\
\end{array}
<br>

**Decision variables:**
\begin{array}{ll}
x_{ij} & \text{Quantity transported between supply node $i \in S$ and demand node $j \in D$}\\
\end{array}
<br>

**Objective:** <br>
$
\begin{align}
  \begin{array}{llll}
    & \min \sum\limits_{i \in S, j \in D} c_{ij} \cdot x_{ij} & &~~~  (1) \\
  \end{array}
\end{align}
$
<br>

**Constraints:**<br>
$
\begin{align}
  \begin{array}{llll}
    & s_i = \sum\limits_{j \in D} x_{ij} &,~ \forall~ i \in S &~~~ (2) \\
    & d_j = \sum\limits_{i \in S} x_{ij} &,~ \forall~ j \in D &~~~ (3) \\
    & x_{ij} \ge 0 &,~ \forall~ i \in S, j \in D &~~~ (4) \\
  \end{array}
\end{align}
$

## Implementation

### Data

In [ ]:
# Supply nodes
S = ['Kiel', 'Magdeburg', 'Dortmund']

# Supply quantity (number of containers)
s = {'Kiel': 45, 'Magdeburg': 120, 'Dortmund': 95}

# Demand nodes
D = ['Hamburg', 'Berlin', 'Cologne', 'Munich']

# Demand quantity (number of containers)
d = {'Hamburg': 80, 'Berlin': 78, 'Cologne': 47, 'Munich': 55}

# Costs (tsd. Euro per container)
c = {
    'Kiel': {'Hamburg': 20, 'Berlin': 40, 'Cologne': 50, 'Munich': 120},
    'Magdeburg': {'Hamburg': 30, 'Berlin': 10, 'Cologne': 40, 'Munich': 60},
    'Dortmund': {'Hamburg': 45, 'Berlin': 40, 'Cologne': 10, 'Munich': 50},
}

### PuLP model

In [ ]:
!pip install pulp
import pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 61.2 MB/s eta 0:00:00


In [ ]:
# Model
model = pulp.LpProblem(name='Classical_Transportation_Problem',
                       sense=pulp.constants.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts(name='x', indices=(S, D), lowBound=0)

# (1) Objective
model += pulp.lpSum(c[i][j] * x[i][j] for i in S for j in D), '(1)'

# (2)
for i in A:
  model += a[i] == pulp.lpSum(x[i][j] for j in D), '(2)_' + str(i)

# (2)
for j in N:
  model += b[j] == pulp.lpSum(x[i][j] for i in S), '(3)_' + str(j)

### Solution

#### Print transport flows

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2), 'tsd. Euro')

# get value of decision variable
for i in S:
  for j in D:
    if x[i][j].varValue > 0:
      print('From ' + str(i) + ' to ' + str(j) + ': ' + str(x[i][j].varValue) + ' container')

Status: Optimal
Objective value: 6020.0 tsd. Euro
From Kiel to Hamburg: 45.0 Container
From Magdeburg to Hamburg: 35.0 Container
From Magdeburg to Berlin: 78.0 Container
From Magdeburg to Munich: 7.0 Container
From Dortmund to Cologne: 47.0 Container
From Dortmund to Munich: 48.0 Container


#### Transport flows as heatmap

In [ ]:
import plotly.express as px

In [ ]:
flow_matrix = [[x[i][j].varValue for j in N] for i in A]

fig = px.imshow(flow_matrix,
                labels=dict(x="Destination", y="Source", color="Flow"),
                x=N,
                y=A,
                text_auto=True,
                color_continuous_scale='Reds'
               )
fig.show()